## Enter full names of group members:

##### Name A: Aaryan Neupane
##### Name B: Anne Torgersen

In [1]:
import math
import numpy as np
from sympy import prime
from pathlib import Path  # for paths of files
import csv
import copy
import random
from sklearn.metrics.pairwise import cosine_similarity

# ANSI escape codes for colors
class colors:
    red = '\033[91m'
    green = '\033[92m'
    blue = '\033[94m'
    end = '\033[0m'  

### 1. DGIM

#### 1.1. DGIM algorithm

In [2]:
# Default DGIM parameters

stream_path = 'data/my_stream.txt'

# The window size
N = 500 

In [3]:
# Helper functions

def shift_buckets(bucket_list):
    for i in range(len(bucket_list) - 1):
        if len(bucket_list[i]) > 2:
            prev_time = bucket_list[i].pop(0)
            penultimate_time = bucket_list[i].pop(0)
            bucket_list[i+1].append(penultimate_time)

def remove_expired(bucket_list, end_time, N):
    for bucket in bucket_list:
        for time_stamp in bucket:
            if (end_time - time_stamp) > N:
                bucket.remove(time_stamp)

In [4]:
def dgim_algorithm(stream_path, N):
    num_buckets = int(N).bit_length()
    bucket_list = [[] for _ in range(num_buckets)]
    time = 0
    with open(stream_path) as file:
        while True:
            bit = file.read(1)

            if not bit:
                break
                
            time += 1

            if bit == '1':
                bucket_list[0].append(time)
                shift_buckets(bucket_list)
                remove_expired(bucket_list, time, N)

    bucket_list = [[elem % N for elem in bucket] for bucket in bucket_list]
            
    end_time = max(bucket_list[0])
    
    return bucket_list, end_time


In [5]:
bucket = dgim_algorithm(stream_path, N)
print(f"The updated list of timestamps buckets from DGIM algorithm: \n {bucket[0]}")
print(f"The end timestamp: {bucket[1]}") 

The updated list of timestamps buckets from DGIM algorithm: 
 [[99], [91, 96], [83, 89], [63, 75], [44], [6], [321, 446], [188], []]
The end timestamp: 99


#### 1.2. Query the Bucket 

In [6]:
def actual_count(stream_path, k):
    stream_list = []
    with open(stream_path, 'r') as file:
        for line in file:
            stream_list.extend(list(map(int, line.strip())))

    # Convert the list into a numpy array
    stream_array = np.array(stream_list)
    
    return int(np.sum(stream_array[-k:]))

In [7]:
def dgim_query(bucket_list, N, k):  
    bucket_list, end_time_stamp = bucket_list
    
    boxes_to_check = 1
    prev_time = 0
    for bucket in bucket_list:
        if len(bucket) == 0:
            break
        if max(bucket) == end_time_stamp:
            continue
            
        time = abs(end_time_stamp - min(bucket))
        if time >= k:
            break
 
        prev_time = abs(end_time_stamp - min(bucket)) + prev_time
        
        if prev_time >= k:
            boxes_to_check += 1
            break
            
        else:
            boxes_to_check += 1
            continue
            
    one_count = 0
    for i in range(boxes_to_check):    
        if i == boxes_to_check:
            if len(bucket_list[i]) == 2:
                one_count += (1 * 2**i) + (1 * 2**i)//2
            elif len(bucket_list[i]) == 1:
                one_count += (1 * 2**i) //2
        one_count += len(bucket_list[i]) * 2**i

    return math.ceil(one_count)

In [8]:
# List of queries
K = [10, 50, 100, 200, 300, 400, 500] 

In [9]:
print("---------------------------------------------------------------")
for k in K:
    dgim_count = dgim_query(bucket, 500, k)
    true_count = actual_count(stream_path, k)
    
    print(f"The total 1s in the last {k} bits by DGIM: {dgim_count}")
    print(f"The true count of 1s in the last {k} bits: {true_count}")
    print(f"The DGIM error for predicted 1s in the last {k} bits: \
    {round(abs(100*(dgim_count-true_count))/true_count,2)} %")
    print("---------------------------------------------------------------")

---------------------------------------------------------------
The total 1s in the last 10 bits by DGIM: 5
The true count of 1s in the last 10 bits: 5
The DGIM error for predicted 1s in the last 10 bits:     0.0 %
---------------------------------------------------------------
The total 1s in the last 50 bits by DGIM: 29
The true count of 1s in the last 50 bits: 26
The DGIM error for predicted 1s in the last 50 bits:     11.54 %
---------------------------------------------------------------
The total 1s in the last 100 bits by DGIM: 45
The true count of 1s in the last 100 bits: 51
The DGIM error for predicted 1s in the last 100 bits:     11.76 %
---------------------------------------------------------------
The total 1s in the last 200 bits by DGIM: 77
The true count of 1s in the last 200 bits: 105
The DGIM error for predicted 1s in the last 200 bits:     26.67 %
---------------------------------------------------------------
The total 1s in the last 300 bits by DGIM: 205
The true c

### 2. Bloom filters

In [10]:
# Username data for the creation of bloom filters - B
data_file = (Path("data/bloom_username").with_suffix('.csv'))

# Test data to check the functionality and false positive rate
test1_file = (Path("data/test1_username").with_suffix('.csv'))
test2_file = (Path("data/test2_username").with_suffix('.csv'))

# Default bloom filter parameters
bloom_size = 1500000 # parameter N
h = 3 # number of hash functions

In [12]:
# create an array of bloom filter with zeros
B = np.zeros(bloom_size)

In [13]:
B

array([0., 0., 0., ..., 0., 0., 0.])

#### 2.1. Create Bloom filter

In [14]:
def generate_hash(h, N):

    hash_list = []
    primes = [n for n in range(2, N) if all(n % x != 0 for x in range(2, int(math.sqrt(n)) + 1))]

    for _ in range(h):
        p = random.choice(primes)
        
        def hash_function(s, p, N):
            hash_value = sum(ord(char) * p**i for i, char in enumerate(s)) % N
            return hash_value

        # Create a lambda function that uses the current p and N
        hf = lambda s: hash_function(s, p, N)
        
        hash_list.append(hf)  # Store the lambda function
                    
    return hash_list 

In [15]:
hashes = generate_hash(h, bloom_size)

In [16]:
def create_bloom_filter(B, hashes, data):
    with data.open() as f:
        for name in f:
            p = random.choice([n for n in range(2, N) if all(n % x != 0 for x in range(2, int(math.sqrt(n)) + 1))])
            for h in hashes:
                hashed_value = sum(ord(char) * p**i for i, char in enumerate(name.strip())) % N
                B[hashed_value] = 1
            
    return B

In [17]:
bloom_array = create_bloom_filter(B, hashes, data_file)

In [18]:
print(bloom_array)

[1. 1. 1. ... 0. 0. 0.]


#### 2.2. Verify usernames

In [19]:
def single_verify_username(bloom_array, hashes, new_user):
    for h in hashes:
        hashed_value = h(new_user)
        if B[hashed_value] == 0:
            code = 0  # Username is definitely available
        else: code = 1  # Username is likely taken
    
    # To-do! verify username and return a code of 0 or 1 (1 - username taken and 0 - username available)
        
    return code
    
    

In [20]:
# Feel free to test different usernames here

new_username = "KazeemTDT4305"

# new_username = "ShambaTDT4305"

In [21]:
user_code = single_verify_username(bloom_array, hashes, new_username)

In [22]:
if user_code == 1:
    print(colors.red + f"Username {new_username} has been taken. Try again!" + colors.end)
elif user_code == 0:
    print(colors.green + f"Username {new_username} is available. Congrats!" + colors.end)
else:
    print(colors.blue + f"Wrong pass code. Please reverify!" + colors.end)  

Username KazeemTDT4305 is available. Congrats!


In [23]:
def group_verify_username(bloom_array, hashes, data):
    # Initialize counts
    total_name = 0
    taken_name = 0
    
    with data.open() as f:
        for name in f:
            total_name += 1
            name = name.strip()  # Remove whitespace
            
            # Check if the username is taken
            if single_verify_username(bloom_array, hashes, name) == 1:
                taken_name += 1
                
    # Calculate and return the percentage of taken usernames
    if total_name == 0:
        return 0
    else:
        return round(taken_name / total_name * 100, 2)

In [24]:
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test1_file)
print(f"Percentage of username seen before from test 1: {user_total}%")
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test2_file)
print(f"Percentage of username seen before from test 2: {user_total}%")
print("----------------------------------------------------------")

----------------------------------------------------------
Percentage of username seen before from test 1: 0.03%
----------------------------------------------------------
Percentage of username seen before from test 2: 0.03%
----------------------------------------------------------


### 3. Flajolet-Martin

In [11]:
def flajolet_martin(input_stream):
    R = 0  # Initialize maximum rightmost zero bit position to 0
    counter = 0 
    
    def h(x):
        hash_output = (6 * x + 1) % 5
        return hash_output
        
    while counter != (len(input_stream)-1):
        
        val = bin(h(input_stream[counter]))[2:]
        count = 0 
        
        for bit in reversed(val):
            if bit == '0':
                count += 1
            else:
                break
                
        if count > R:
            R = count
        counter += 1
        
    distinct_estimate = 2**R

    return distinct_estimate

In [12]:
# Input stream
input_stream1 = [1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1]
input_stream2 = [1, 3, 2, 1, 2, 3, 4, 3, 1, 2, 3, 1]

# Run the Flajolet-Martin algorithm
distinct_estimate1 = flajolet_martin(input_stream1)
distinct_estimate2 = flajolet_martin(input_stream2)

# Print the estimated number of distinct elements
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 1:", distinct_estimate1)
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 2:", distinct_estimate2)
print("-----------------------------------------------------")

-----------------------------------------------------
Distinct elements (estimated) in input stream 1: 2
-----------------------------------------------------
Distinct elements (estimated) in input stream 2: 4
-----------------------------------------------------


### 4. Adword 

#### 4.1. Greedy Algorithm

In [13]:
# User queries
queries = ["big data", "big data", "big data","bloom filters", "bloom filters", "bloom filters",
           "flajolet martin", "flajolet martin", "flajolet martin", "dgim algorithm", "dgim algorithm", "dgim algorithm"]

In [14]:
# Company A B C and D keywords and budget $$$
global_companies = {
        'A': ["big data", "bloom filters", 3],
        'B': ["flajolet martin", 3],
        'C': ["flajolet martin", "dgim algorithm", 3],
        'D': ["big data", 3],
    }

In [15]:
def greedy_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0     
    for query in queries:
        # Collect companies that have bid for the query and have a positive budget
        available_companies = [company for company, company_data in local_companies.items() if query in company_data[:-1] and company_data[-1] > 0]
        
        if available_companies:
            # Randomly select one company
            selected_company = random.choice(available_companies)
            
            # Increment revenue and decrement the selected company's budget
            revenue += 1
            local_companies[selected_company][-1] -= 1  # Decrement the budget of the selected company
            
    return revenue

In [16]:
total_revenue = 0
total_trials = 10
print("Starting trials using Greedy Algorithm...")
print("------------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = greedy_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("------------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Greedy Algorithm...
------------------------------------------------
Trial 1 - Revenue generated: 10
Trial 2 - Revenue generated: 8
Trial 3 - Revenue generated: 9
Trial 4 - Revenue generated: 9
Trial 5 - Revenue generated: 7
Trial 6 - Revenue generated: 9
Trial 7 - Revenue generated: 10
Trial 8 - Revenue generated: 10
Trial 9 - Revenue generated: 11
Trial 10 - Revenue generated: 10
------------------------------------------------
Average revenue generated for all trials:  9.3


#### 4.2. Balance Algorithm

In [17]:
def balance_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    for query in queries:
        # Collect companies that have bid for the query and have a positive budget
        available_companies = [company for company, company_data in local_companies.items() if query in company_data[:-1] and company_data[-1] > 0]
        
        if available_companies:
            # Select the advertiser with the largest remaining budget for the query
            max_budget = max([local_companies[company][-1] for company in available_companies])
            tied_companies = [company for company in available_companies if local_companies[company][-1] == max_budget]
            
            # Randomly select one company from the tied companies
            selected_company = random.choice(tied_companies)
            
            # Increment revenue and decrement the selected company's budget
            revenue += 1
            local_companies[selected_company][-1] -= 1  # Decrement the budget of the selected company
    
    return revenue

In [18]:
total_revenue = 0
total_trials = 10
print("Starting trials using Balance Algorithm...")
print("-------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = balance_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("-------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Balance Algorithm...
-------------------------------------------
Trial 1 - Revenue generated: 8
Trial 2 - Revenue generated: 9
Trial 3 - Revenue generated: 10
Trial 4 - Revenue generated: 9
Trial 5 - Revenue generated: 9
Trial 6 - Revenue generated: 8
Trial 7 - Revenue generated: 8
Trial 8 - Revenue generated: 10
Trial 9 - Revenue generated: 9
Trial 10 - Revenue generated: 9
-------------------------------------------
Average revenue generated for all trials:  8.9


### 5. Recommender System

In [19]:
# Ratings matrix (each row corresponds to a movie, and each column corresponds to a user)
ratings_matrix = np.array([
    [1, 0, 3, 0, 0, 5, 0, 0, 5, 0, 4, 0],
    [0, 0, 5, 4, 0, 0, 4, 0, 0, 2, 1, 3],
    [2, 4, 0, 1, 2, 0, 3, 0, 4, 3, 5, 0],
    [0, 2, 4, 0, 5, 0, 0, 4, 0, 0, 2, 0],
    [0, 0, 4, 3, 4, 2, 0, 0, 0, 0, 2, 5],
    [1, 0, 3, 0, 3, 0, 0, 2, 0, 0, 4, 0]
])

#### 5.1. User-User Collaborative Filtering

In [20]:
# Helper functions

def cos_sim(user1_v, user2_v):
    return np.dot(user1_v, user2_v) / (np.linalg.norm(user1_v) * np.linalg.norm(user2_v))
    
def get_top_indices(arr, x):
    sorted_indices = np.argsort(arr)
    # Find the highest indices, ignoring the first
    top_indices = sorted_indices[-(x+1):-1]
    return top_indices

In [21]:
def user_cf(rate_m, tup_mu, neigh):
    movie_id, user_id = tup_mu
    
    user_v = rate_m.T[user_id - 1]

    similarities = []

    for user in rate_m.T:
        similarities.append(cos_sim(user_v, user))

    N_users = get_top_indices(similarities, neigh)

    pred_numer = 0
    pred_denom = 0

    for user in N_users:
        # Weighted average
        pred_numer += similarities[user] * rate_m[movie_id - 1][user]
        pred_denom += similarities[user]

    prediction = round(pred_numer / pred_denom, 2)    
    
    return prediction

In [22]:
# List of tuple of movie rating by users to be predicted e.g (1, 5) refers to the rating of movie 1 by user 5
list_mu_query = [(1, 5), (3, 3)]

# Neighbor selection (|N|)
neigh = 2

In [23]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = user_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (User-User CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 1.42 (User-User CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 1.49 (User-User CF)
-----------------------------------------------------------------


#### 5.2. Item-Item Collaborative Filtering

In [24]:
def item_cf(rate_m, tup_mu, neigh):
    movie_id, user_id = tup_mu

    movie_v = rate_m[movie_id - 1]
    user_v = rate_m.T[user_id - 1]

    similarities = []

    for movie in rate_m:
        similarities.append(cos_sim(movie_v, movie))

    N_movies = get_top_indices(similarities, neigh)

    pred_numer = 0
    pred_denom = 0

    for movie in N_movies:
        # Weighted average
        pred_numer += similarities[movie] * user_v[movie]
        pred_denom += similarities[movie]
        
    prediction = round(pred_numer / pred_denom, 2)
    
    return prediction

In [25]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = item_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (Item-Item CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 2.48 (Item-Item CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 3.0 (Item-Item CF)
-----------------------------------------------------------------


### Provide concise answers to all 5 cases in the Project 3 description below

#### Case 1

The space complexity of DGIM is $O(\log^2 N)$ because each bucket can represent up to $\log N$ timestamps, resulting in a total of $log^2 N$ timestamps across all buckets. 


#### Case 2

Yes. Bloom filters can produce false positives, but not false negatives. 

Case presentation for the site admin: When the bloom filter algorithm returns that a username is taken, it actually means that username is most likely taken. This is because bloom filters are a probabilistic data structure. The way false positives are generated is through hash collisions. This is the case when two usernames hashes to the same value. A hash collision happens often, but for the bloom filter to return a false positive, the two usernames needs to hash to the same value for all the hash values. Therefore, to prevent two usernames to hash to the same values, we use multiple and robust hash functions. 

I have to disagree with you on this one, my friend. For me to be able to create a user with this username, it is simply not possible for this username to already exist. A bloom filter can not produce false negatives. The only way this would be possible, is if some of the hash functions produce different hash values from the same output. Since hash functions needs to be deterministic, this is simply not possible. And if you did not now - deterministic means that same input should always produce the same output.

#### Case 3

The main improvement area is the hashing process, and the way to increase precision is to use a better hash function or increase the number of hash functions. A better hash function means a function that generate more random and diverse hash values. By increasing the number of hash functions you can find the average of the estimates from different hash functions, thus making the final approximation more accurate.

#### Case 4

Greedy algorithm:

Minimum revenue: 6

Maximum revenue: 12

Competitiveness: 6/12 = 1/2

Balance algorithm:

Minimum revenue: 8

Maximum revenue: 10

Competitveness: 8/12 = 2/3

#### Case 5

By analyzing the ratings matrix, it's clear that users 3 and 5 share similar tastes in the provided movies. Since user 3 rated movie 1 a 3, it is natural to assume that a prediction closer to 3 would be more accurate. Based on this intuition, item-item collaborative filtering seems like the better prediction.